In [1]:
import socket
socket.gethostname()

'calebs-instance-2'

In [2]:
import gcsfs

In [3]:
fs = gcsfs.GCSFileSystem(project='mvdmlab', token='cloud')

In [4]:
fs.ls('motivational-t/R042')

['motivational-t/R042/R042-2013-08-16.zip',
 'motivational-t/R042/R042-2013-08-17.zip',
 'motivational-t/R042/R042-2013-08-18/',
 'motivational-t/R042/R042-2013-08-19.zip',
 'motivational-t/R042/R042-2013-08-19/',
 'motivational-t/R042/R042-2013-08-20.zip',
 'motivational-t/R042/R042-2013-08-21.zip']

In [5]:
import os
import sys
import numpy as np
import nept
import matplotlib.pyplot as plt


In [6]:

# define some testing data file locations
fd = 'motivational-t/R042/R042-2013-08-18/' # need function that makes list of all folders
fn_lfp = 'R042-2013-08-18-CSC11a.ncs' # from *keys.m, need loader to make file into dict
fn_vt = 'VT1.nvt'
CONST_conv_fact = [2.9176, 2.3794] # pixels to cm conversion for x, y (also from ExpKeys)

# fix annoying 'display last result only' behavior
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# load and show lfp using nept loaders
lfp = nept.load_lfp(os.path.join(fd,fn_lfp))

start = 4261
stop = 4262
lfpR = lfp.time_slice(start, stop)

#plt.plot(lfp.time, lfp.data)
plt.plot(lfpR.time, lfpR.data)
plt.show()

In [ ]:
# Load position (.nvt) from this experiment
position = nept.load_position(os.path.join(fd, fn_vt),CONST_conv_fact)

# Plot the position
plt.plot(position.x, position.y, 'k.', ms=1)
plt.show()

In [ ]:
# load spikes
spikes = nept.load_spikes(fd)

# Plot the spikes
for idx, spiketrain in enumerate(spikes):
    plt.plot(spiketrain.time, np.ones(len(spiketrain.time))+idx, '|')
plt.show()

In [ ]:
# construct corresponding objects in nelpy
import nelpy as nel

In [ ]:
# lfp should be AnalogSignalArray
lfp_nelpy = nel.AnalogSignalArray(ydata=lfp.data,timestamps=lfp.time,merge_sample_gap=1)
print(lfp_nelpy)

In [ ]:
# vt is a 2-D AnalogSignalArray -- note has bigger gaps in data b/c tracking LEDs get obscured
vt_nelpy = np.stack((position.x,position.y),1)
vt_nelpy.shape
vt_nelpy = nel.AnalogSignalArray(ydata=vt_nelpy.T,timestamps=position.time,merge_sample_gap=10)
print(vt_nelpy)

In [ ]:
# load candidates
fn_cand = 'R042-2013-08-18-candidates.mat'
import scipy.io as sio
cand = sio.loadmat(os.path.join(fd,fn_cand))

In [ ]:
tstart = cand['evt'][0][0]['tstart']
tend = cand['evt'][0][0]['tend']